In [ ]:
%matplotlib inline
'''
How to Optimize Your Dragon: V1
Import initialized model and start the optimization
'''
# system imports
import sys
import os
import datetime
sys.path.append(os.path.abspath('..'))
sys.path.append(os.path.abspath('../..'))

import numpy as np
from matplotlib import pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

import dill

# pyomo imports
from pyomo import environ as pe
from global_sets.component import m

from stages.reactive_stage import reactive_stage_rule
from stages.condenser_stage import condenser_stage_rule
from stages.reboiler_stage import reboiler_stage_rule

from utility.display_utility import beautify, beautify_reactive, HiddenLogs, HiddenPrints, plot_distribution,\
                                    trans_cnumber, trans_product_mole
from utility.model_utility import add_dual, update_dual, delete_dual, check_DOF# , check_violate_constraint
from utility.data_utility import cal_cnumber
from utility.time_utility import create_filename_time, log_now, log_end

In [ ]:
product = {}
product['napha'] = [i for i in m.COMP_ORG if cal_cnumber(i) >= 5 and cal_cnumber(i) <= 7]
product['gasoline'] = [i for i in m.COMP_ORG if cal_cnumber(i) >= 8 and cal_cnumber(i) <= 12]
product['diesel'] = [i for i in m.COMP_ORG if cal_cnumber(i) >= 13 and cal_cnumber(i) <= 18]
product['heavy'] = [i for i in m.COMP_ORG if cal_cnumber(i) >= 19 and cal_cnumber(i) <= 56]

In [ ]:
logname = create_filename_time()
log_text_dir = './log/text/opt_'+logname+'.dat'
log_figure_dir = './log/figure/opt_'+logname+'.pdf'

In [ ]:
def see_product(model):
    cd_x_data = []; rf_x_data = {}; rb_x_data = []

    cd_x_data = trans_product_mole({i:model.condenser.x[i].value for i in m.COMP_TOTAL})
    for j in model.reactive:
        rf_x_data[j] = trans_product_mole({i:model.reactive[j].x[i].value for i in m.COMP_TOTAL})
    rb_x_data = trans_product_mole({i:model.reboiler.x[i].value for i in m.COMP_TOTAL})

    print('Napha:\t\t{:.2f}'.format(cd_x_data['scaled']['napha']))
    print('Gasoline:\t{:.2f}'.format(rf_x_data[7]['scaled']['gasoline']))
    print('Diesel:\t\t{:.2f}'.format(rf_x_data[10]['scaled']['diesel']))
    print('Heavy:\t\t{:.2f}'.format(rb_x_data['scaled']['heavy']))

In [ ]:
with open('./log/model/stage_20_base.pickle','rb') as f:
    model = dill.load(f)

In [ ]:
# for j in model.reactive:
#     model.reactive[j].MPCC.pf.fix(0)
# model.reboiler.MPCC.pf.fix(0)

In [ ]:
opt = pe.SolverFactory('ipopt')

opt.options['print_user_options'] = 'yes'
opt.options['linear_solver'] = 'ma86'

opt.options['linear_system_scaling '] = 'mc19'
opt.options['linear_scaling_on_demand '] = 'no'

opt.options['max_iter'] = 7000
opt.options['warm_start_init_point'] = 'yes'
opt.options['warm_start_bound_push'] = 1e-20
opt.options['warm_start_mult_bound_push'] = 1e-20
opt.options['mu_init'] = 1e-6

In [ ]:
results = opt.solve(model,tee=True)
update_dual(pe,model)

In [ ]:
pdf = PdfPages(log_figure_dir)

In [ ]:
with HiddenLogs(log_text_dir,'w'):
    print('\n>','Original 20 stage case')
    print('-'*108)
    beautify(pe,model)
    see_product(model)
    log_now()

plot_distribution(model,pdf,'Original 20 stage case')

# Optimization Input

**Parameters:**
* Stage Temperature
    * Rectifying section: 200C - 300C
    * Stripping section: 200C - 300C
* Reflux
    * Distillate / (Distillate + Reflux): 0.5 - 0.05, Refulx Ratio: 1 - 19


**Constraints:**
* Product
    * Distillate: C5~C7 >= 0.75
    * Gasoline: C8~C12 >= 0.75
    * Diesel: C13~C18 >= 0.6
    * Heavy: C19+ >= 0.85
    
**Objective:**
* Max gasoline production

### Parameters

In [ ]:
for j in range(8,13):
    model.reactive[j].T.setlb(200+273.15)
    model.reactive[j].T.setub(300+273.15)
for j in range(14,19):
    model.reactive[j].T.setlb(200+273.15)
    model.reactive[j].T.setub(300+273.15)
    
# model.condenser.PR_L.unfix()
# model.condenser.PR_L.setlb(0.05)
# model.condenser.PR_L.setub(0.5)

### Constraints

In [ ]:
model.quality_coefficient = pe.Param(within=pe.NonNegativeReals,mutable=True,initialize=1)

In [ ]:
model.product_spec_con = pe.ConstraintList()

In [ ]:
model.product_spec_con.add(expr = sum(model.condenser.x[i] for i in product['napha']) >= \
                           model.quality_coefficient*0.75*sum(model.condenser.x[i] for i in m.COMP_ORG));
model.product_spec_con.add(expr = sum(model.reactive[7].x[i] for i in product['gasoline']) >= \
                           model.quality_coefficient*0.75*sum(model.reactive[7].x[i] for i in m.COMP_ORG));
model.product_spec_con.add(expr = sum(model.reactive[10].x[i] for i in product['diesel']) >= \
                           model.quality_coefficient*0.6*sum(model.reactive[10].x[i] for i in m.COMP_ORG));
model.product_spec_con.add(expr = sum(model.reboiler.x[i] for i in product['heavy']) >= \
                           model.quality_coefficient*0.85*sum(model.reboiler.x[i] for i in m.COMP_ORG));

### Objective

In [ ]:
model.del_component(model.obj)

In [ ]:
model.obj = pe.Objective(expr = model.reactive[7].L['P'] - sum(model.reactive[j].MPCC.pf for j in model.reactive)\
                                - model.reboiler.MPCC.pf, sense = pe.maximize)

In [ ]:
# increase the penalty weight
# for j in model.reactive:
#     model.reactive[j].MPCC.rho = 100
# model.reboiler.MPCC.rho = 100

In [ ]:
results = opt.solve(model,tee=True)
update_dual(pe,model)

In [ ]:
with HiddenLogs(log_text_dir):
    print('\n>','Optimized Temperature and Reflux')
    print('-'*108)
    beautify(pe,model)
    see_product(model)
    log_now()

plot_distribution(model,pdf,'Optimized Temperature')

In [ ]:
model.condenser.PR_L.unfix()
model.condenser.PR_L.setlb(0.05)
model.condenser.PR_L.setub(0.5)

In [ ]:
results = opt.solve(model,tee=True)
update_dual(pe,model)

In [ ]:
with HiddenLogs(log_text_dir):
    print('\n>','Optimized Temperature and Reflux')
    print('-'*108)
    beautify(pe,model)
    see_product(model)
    log_now()

plot_distribution(model,pdf,'Optimized Reflux')

# Open up side-draw

**Parameters:**
* Stage Temperature
    * Rectifying section: 200C - 300C
    * Stripping section: 200C - 300C
* Reflux
    * Distillate / (Distillate + Reflux): 0.5 - 0.05, Refulx Ratio: 1 - 19
* Side-draw ratio
    * PR_L: 0 - 1

**Constraints:**
* Product
    * Distillate: C5~C7 >= 0.75
    * Gasoline: C8~C12 >= 0.75
    * Diesel: C13~C18 >= 0.6
    * Heavy: C19+ >= 0.85
    
**Objective:**
* Max gasoline production

In [ ]:
model.reactive[4].PR_L.unfix()
model.reactive[4].L['P'].setlb(0.02)
model.reactive[7].PR_L.unfix()
model.reactive[10].PR_L.unfix()

In [ ]:
results = opt.solve(model,tee=True)
update_dual(pe,model)

In [ ]:
with HiddenLogs(log_text_dir):
    print('\n>','Optimized Product Side Draw')
    print('-'*108)
    beautify(pe,model)
    see_product(model)
    log_now()

plot_distribution(model,pdf,'Optimized Product Side Draw')

# Open up feed allocation

**Parameters:**
* Stage Temperature
    * Rectifying section: 200C - 300C
    * Stripping section: 200C - 300C
* Reflux
    * Distillate / (Distillate + Reflux): 0.5 - 0.05, Refulx Ratio: 1 - 19
* Side-draw
    * PR_L: 0 - 1
    
* Feed
    * F: 0 - 3

**Constraints:**
* Product
    * Distillate: C5~C7 >= 0.75
    * Gasoline: C8~C12 >= 0.75
    * Diesel: C13~C18 >= 0.6
    * Heavy: C19+ >= 0.85
    
* Total feed
    * Total feed = 10 kmol/s
    
**Objective:**
* Max gasoline production

In [ ]:
for j in model.reactive:
    model.reactive[j].F.unfix()
    model.reactive[j].F.setlb(0)
    model.reactive[j].F.setub(10)

In [ ]:
model.total_feed_con = pe.ConstraintList()

In [ ]:
model.total_feed_con.add(expr = sum(model.reactive[j].F for j in model.reactive) == 10);

In [ ]:
results = opt.solve(model,tee=True)
update_dual(pe,model)

In [ ]:
with HiddenLogs(log_text_dir):
    print('\n>','Optimized Feed Amount and Location')
    print('-'*108)
    beautify(pe,model)
    see_product(model)
    log_now()

plot_distribution(model,pdf,'Optimized Feed Amount and Location')

# Increase spec a little bit

**Parameters:**
* Stage Temperature
    * Rectifying section: 200C - 300C
    * Stripping section: 200C - 300C
* Reflux
    * Distillate / (Distillate + Reflux): 0.5 - 0.05, Refulx Ratio: 1 - 19
* Side-draw
    * PR_L: 0 - 1
    
* Feed
    * F: 0 - 3

**Constraints:**
* Product
    * Distillate: C5~C7 >= 0.75
    * Gasoline: C8~C12 >= **0.8**
    * Diesel: C13~C18 >= 0.6
    * Heavy: C19+ >= 0.85
    
* Total feed
    * Total feed = 10 kmol/s
    
**Objective:**
* Max gasoline production

In [ ]:
# model.product_spec_con.add(expr = sum(model.reactive[7].x[i] for i in product['gasoline']) >= \
#                            model.quality_coefficient*0.8*sum(model.reactive[7].x[i] for i in m.COMP_ORG));

In [ ]:
# results = opt.solve(model,tee=True)
# update_dual(pe,model)

In [ ]:
# with HiddenLogs(log_text_dir):
#     print('\n>','Increase Gasoline Specification to 80%')
#     print('-'*108)
#     beautify(pe,model)
#     see_product(model)
#     log_now()

# plot_distribution(model,pdf,'Increase Gasoline Specification to 80%')

# Open up catalyst allocation ( = TOTAL reconstruction, long wait time)

**Parameters:**
* Stage Temperature
    * Rectifying section: 200C - 300C
    * Stripping section: 200C - 300C
* Reflux
    * Distillate / (Distillate + Reflux): 0.5 - 0.05, Refulx Ratio: 1 - 19
* Side-draw
    * PR_L: 0 - 1
    
* Feed
    * F: 0 - 3
        
* Catalyst
    * cat: 0 - 10000

**Constraints:**
* Product
    * Distillate: C5~C7 >= 0.75
    * Gasoline: C8~C12 >= 0.75 / 0.8
    * Diesel: C13~C18 >= 0.6
    * Heavy: C19+ >= 0.85
    
* Total feed
    * sum feed = 10 kmol/s
    
* Total catalyst
    * sum cat = 30000 kg
    
**Objective:**
* Max gasoline production

In [ ]:
for j in model.reactive:
    model.reactive[j].cat.unfix()
    model.reactive[j].cat.setlb(0)
    model.reactive[j].cat.setub(30000)

In [ ]:
model.total_cat_con = pe.ConstraintList()

In [ ]:
model.total_cat_con.add(expr = sum(model.reactive[j].cat for j in model.reactive) == 10*3000);

In [ ]:
results = opt.solve(model,tee=True)
update_dual(pe,model)

In [ ]:
with HiddenLogs(log_text_dir):
    print('\n>','Optimized Catalyst Amount and Location')
    print('-'*108)
    beautify(pe,model)
    see_product(model)
    log_now()

plot_distribution(model,pdf,'Optimized Catalyst Amount and Location')

In [ ]:
pdf.close()